In [1]:
import sqlite3
from sqlite3 import OperationalError

conn = sqlite3.connect('./data/NBAdatabase.db')

In [2]:
import pandas as pd
raw_data = pd.read_csv("raw/2022.txt")
raw_data = raw_data.rename({'Start (ET)': 'start','Visitor/Neutral': 'visitor', 'PTS': 'visitor_pts','Home/Neutral': 'home', 'PTS.1': 'home_pts'}, axis=1)
raw_data = raw_data.drop(['Unnamed: 6', 'Unnamed: 7'], axis=1)
raw_data

,Date,start,visitor,visitor_pts,home,home_pts,Attend.,Notes
0,Tue Oct 19 2021,7:30p,Brooklyn Nets,104,Milwaukee Bucks,127,17341.0,NaN
1,Tue Oct 19 2021,10:00p,Golden State Warriors,121,Los Angeles Lakers,114,18997.0,NaN
2,Wed Oct 20 2021,7:00p,Indiana Pacers,122,Charlotte Hornets,123,15521.0,NaN
3,Wed Oct 20 2021,7:00p,Chicago Bulls,94,Detroit Pistons,88,20088.0,NaN
4,Wed Oct 20 2021,7:30p,Boston Celtics,134,New York Knicks,138,19812.0,NaN
...,...,...,...,...,...,...,...,...
1145,Thu Mar 31 2022,7:00p,Philadelphia 76ers,94,Detroit Pistons,102,20023.0,NaN
1146,Thu Mar 31 2022,7:30p,Cleveland Cavaliers,107,Atlanta Hawks,131,17491.0,NaN
1147,Thu Mar 31 2022,7:30p,Milwaukee Bucks,120,Brooklyn Nets,119,17917.0,NaN
1148,Thu Mar 31 2022,8:00p,Los Angeles Clippers,130,Chicago Bulls,135,21519.0,NaN


In [3]:
raw_data["Date"]= pd.to_datetime(raw_data["Date"])
raw_data['start'] = pd.to_datetime(raw_data['start'])
raw_data['start'] = [time.time() for time in raw_data['start']]
raw_data

,Date,start,visitor,visitor_pts,home,home_pts,Attend.,Notes
0,2021-10-19,19:30:00,Brooklyn Nets,104,Milwaukee Bucks,127,17341.0,NaN
1,2021-10-19,22:00:00,Golden State Warriors,121,Los Angeles Lakers,114,18997.0,NaN
2,2021-10-20,19:00:00,Indiana Pacers,122,Charlotte Hornets,123,15521.0,NaN
3,2021-10-20,19:00:00,Chicago Bulls,94,Detroit Pistons,88,20088.0,NaN
4,2021-10-20,19:30:00,Boston Celtics,134,New York Knicks,138,19812.0,NaN
...,...,...,...,...,...,...,...,...
1145,2022-03-31,19:00:00,Philadelphia 76ers,94,Detroit Pistons,102,20023.0,NaN
1146,2022-03-31,19:30:00,Cleveland Cavaliers,107,Atlanta Hawks,131,17491.0,NaN
1147,2022-03-31,19:30:00,Milwaukee Bucks,120,Brooklyn Nets,119,17917.0,NaN
1148,2022-03-31,20:00:00,Los Angeles Clippers,130,Chicago Bulls,135,21519.0,NaN


In [4]:
from sklearn.preprocessing import LabelEncoder

raw_data['datetime'] = pd.to_datetime(raw_data["Date"].astype(str) + " " + raw_data["start"].astype(str))



In [5]:
total_lst = []
for row in raw_data.index:
    total_lst.append((raw_data['datetime'][row], 
                           raw_data['home'][row], 
                           raw_data['home_pts'][row], 
                           raw_data['visitor'][row], 
                           raw_data['visitor_pts'][row] ))
new_lst = []
name_lst = []
for row in total_lst:
    if row[0].day <=20 and row[0].month == 10:
        new_lst.append(row)
    if row[1] not in name_lst:
        name_lst.append(row[1])
    if row[3] not in name_lst:
        name_lst.append(row[3])
total_lst = new_lst

le =  LabelEncoder()
encoded_labels = le.fit_transform(name_lst)
labels_dict = {}
for i in range(len(name_lst)):
    labels_dict[name_lst[i]] = encoded_labels[i]
    
print(labels_dict)


{'Milwaukee Bucks': 16, 'Brooklyn Nets': 2, 'Los Angeles Lakers': 13, 'Golden State Warriors': 9, 'Charlotte Hornets': 3, 'Indiana Pacers': 11, 'Detroit Pistons': 8, 'Chicago Bulls': 4, 'New York Knicks': 19, 'Boston Celtics': 1, 'Toronto Raptors': 27, 'Washington Wizards': 29, 'Memphis Grizzlies': 14, 'Cleveland Cavaliers': 5, 'Minnesota Timberwolves': 17, 'Houston Rockets': 10, 'New Orleans Pelicans': 18, 'Philadelphia 76ers': 22, 'San Antonio Spurs': 26, 'Orlando Magic': 21, 'Utah Jazz': 28, 'Oklahoma City Thunder': 20, 'Portland Trail Blazers': 24, 'Sacramento Kings': 25, 'Phoenix Suns': 23, 'Denver Nuggets': 7, 'Atlanta Hawks': 0, 'Dallas Mavericks': 6, 'Miami Heat': 15, 'Los Angeles Clippers': 12}


 Create Table "Team"

In [6]:
# conn.execute('''DROP TABLE team;''')
conn.execute('''CREATE TABLE team(
                TeamId INTEGER NOT NULL,
                Name VARCHAR,
                PRIMARY KEY (TeamId)
                )''')

for ele in labels_dict:
    conn.execute( ''' INSERT INTO team(TeamId, Name) VALUES(?,?) ''', (int(labels_dict[ele]), ele))

In [7]:
results = list(conn.execute( ''' SELECT * FROM team'''))
print(results)

[(0, 'Atlanta Hawks'), (1, 'Boston Celtics'), (2, 'Brooklyn Nets'), (3, 'Charlotte Hornets'), (4, 'Chicago Bulls'), (5, 'Cleveland Cavaliers'), (6, 'Dallas Mavericks'), (7, 'Denver Nuggets'), (8, 'Detroit Pistons'), (9, 'Golden State Warriors'), (10, 'Houston Rockets'), (11, 'Indiana Pacers'), (12, 'Los Angeles Clippers'), (13, 'Los Angeles Lakers'), (14, 'Memphis Grizzlies'), (15, 'Miami Heat'), (16, 'Milwaukee Bucks'), (17, 'Minnesota Timberwolves'), (18, 'New Orleans Pelicans'), (19, 'New York Knicks'), (20, 'Oklahoma City Thunder'), (21, 'Orlando Magic'), (22, 'Philadelphia 76ers'), (23, 'Phoenix Suns'), (24, 'Portland Trail Blazers'), (25, 'Sacramento Kings'), (26, 'San Antonio Spurs'), (27, 'Toronto Raptors'), (28, 'Utah Jazz'), (29, 'Washington Wizards')]


In [9]:
# conn.execute('''DROP TABLE schedule;''')
conn.execute('''CREATE TABLE schedule(
                GameId INTEGER NOT NULL,
                date TEXT NOT NULL,
                homeId INTEGER,
                home_pts INTEGER,
                visitorId INTEGER,
                visitor_pts INTEGER,
                FOREIGN KEY ([homeId]) REFERENCES [team] ([TeamId]) ON DELETE NO ACTION ON UPDATE NO ACTION,
                FOREIGN KEY ([visitorId]) REFERENCES [team] ([TeamId]) ON DELETE NO ACTION ON UPDATE NO ACTION,
                PRIMARY KEY (GameId)
                )''')

for num, ele in enumerate(total_lst):
    dt = ele[0].strftime('%Y-%m-%d %H:%M:%S')
    conn.execute( ''' INSERT INTO schedule(GameId, date, homeId, home_pts, visitorId, visitor_pts) VALUES(?,?,?,?,?,?) ''', (num, dt, int(labels_dict[ele[1]]), int(ele[2]), int(labels_dict[ele[3]]), int(ele[4])))

In [10]:
results = list(conn.execute( ''' SELECT * FROM schedule'''))
print(results)

[(0, '2021-10-19 19:30:00', 16, 127, 2, 104), (1, '2021-10-19 22:00:00', 13, 114, 9, 121), (2, '2021-10-20 19:00:00', 3, 123, 11, 122), (3, '2021-10-20 19:00:00', 8, 88, 4, 94), (4, '2021-10-20 19:30:00', 19, 138, 1, 134), (5, '2021-10-20 19:30:00', 27, 83, 29, 98), (6, '2021-10-20 20:00:00', 14, 132, 5, 121), (7, '2021-10-20 20:00:00', 17, 124, 10, 106), (8, '2021-10-20 20:00:00', 18, 97, 22, 117), (9, '2021-10-20 20:30:00', 26, 123, 21, 97), (10, '2021-10-20 21:00:00', 28, 107, 20, 86), (11, '2021-10-20 22:00:00', 24, 121, 25, 124), (12, '2021-10-20 22:00:00', 23, 98, 7, 110)]


In [11]:
import os

root = r"raw/new/"
file_list = [os.path.join(path, name) for path, subdirs, files in os.walk(root) for name in files]
print(file_list)

['raw/new/19BN_adv.csv', 'raw/new/19BN_basic.csv', 'raw/new/19GW_adv.csv', 'raw/new/19GW_basic.csv', 'raw/new/19LL_adv.csv', 'raw/new/19LL_basic.csv', 'raw/new/19MB_adv.csv', 'raw/new/19MB_basic.csv', 'raw/new/20BC_adv.csv', 'raw/new/20BC_basic.csv', 'raw/new/20CB_adv.csv', 'raw/new/20CB_basic.csv', 'raw/new/20CCbasic.csv', 'raw/new/20CC_adv.csv', 'raw/new/20CH_adv.csv', 'raw/new/20CH_basic.csv', 'raw/new/20DN_adv.csv', 'raw/new/20DN_basic.csv', 'raw/new/20DP_adv.csv', 'raw/new/20DP_basic.csv', 'raw/new/20HR_adv.csv', 'raw/new/20HR_basic.csv', 'raw/new/20IP_adv.csv', 'raw/new/20IP_basic.csv', 'raw/new/20MG_adv.csv', 'raw/new/20MG_basic.csv', 'raw/new/20MT_adv.csv', 'raw/new/20MT_basic.csv', 'raw/new/20NK_adv.csv', 'raw/new/20NK_basic.csv', 'raw/new/20NP_adv.csv', 'raw/new/20NP_basic.csv', 'raw/new/20OM_adv.csv', 'raw/new/20OM_basic.csv', 'raw/new/20OT_adv.csv', 'raw/new/20OT_basic.csv', 'raw/new/20P7_adv.csv', 'raw/new/20P7_basic.csv', 'raw/new/20PB_adv.csv', 'raw/new/20PB_basic.csv', 

In [12]:
import pandas as pd
player_lst = []
print(file_list)

# df = pd.read_csv('raw/new/20BC_adv.csv')
# df

for file in file_list:
    df = pd.read_csv(file)
    for index, row in df.iterrows():
        if row['Unnamed: 0'] != 'Starters'and row['Unnamed: 0'] not in player_lst:
            player_lst.append(row['Unnamed: 0'])

print(len(player_lst))
player_dict = {}
for idx, name in enumerate(player_lst):
    player_dict[name] = idx
    
player_dict

['raw/new/19BN_adv.csv', 'raw/new/19BN_basic.csv', 'raw/new/19GW_adv.csv', 'raw/new/19GW_basic.csv', 'raw/new/19LL_adv.csv', 'raw/new/19LL_basic.csv', 'raw/new/19MB_adv.csv', 'raw/new/19MB_basic.csv', 'raw/new/20BC_adv.csv', 'raw/new/20BC_basic.csv', 'raw/new/20CB_adv.csv', 'raw/new/20CB_basic.csv', 'raw/new/20CCbasic.csv', 'raw/new/20CC_adv.csv', 'raw/new/20CH_adv.csv', 'raw/new/20CH_basic.csv', 'raw/new/20DN_adv.csv', 'raw/new/20DN_basic.csv', 'raw/new/20DP_adv.csv', 'raw/new/20DP_basic.csv', 'raw/new/20HR_adv.csv', 'raw/new/20HR_basic.csv', 'raw/new/20IP_adv.csv', 'raw/new/20IP_basic.csv', 'raw/new/20MG_adv.csv', 'raw/new/20MG_basic.csv', 'raw/new/20MT_adv.csv', 'raw/new/20MT_basic.csv', 'raw/new/20NK_adv.csv', 'raw/new/20NK_basic.csv', 'raw/new/20NP_adv.csv', 'raw/new/20NP_basic.csv', 'raw/new/20OM_adv.csv', 'raw/new/20OM_basic.csv', 'raw/new/20OT_adv.csv', 'raw/new/20OT_basic.csv', 'raw/new/20P7_adv.csv', 'raw/new/20P7_basic.csv', 'raw/new/20PB_adv.csv', 'raw/new/20PB_basic.csv', 

{'Joe Harris': 0,
 'James Harden': 1,
 'Kevin Durant': 2,
 'Nic Claxton': 3,
 'Blake Griffin': 4,
 'Patty Mills': 5,
 'James Johnson': 6,
 'Jevon Carter': 7,
 'LaMarcus Aldridge': 8,
 'Paul Millsap': 9,
 "DeAndre' Bembry": 10,
 'Bruce Brown': 11,
 'Cam Thomas': 12,
 'Kessler Edwards': 13,
 'David Duke Jr.': 14,
 'Stephen Curry': 15,
 'Draymond Green': 16,
 'Andrew Wiggins': 17,
 'Jordan Poole': 18,
 'Kevon Looney': 19,
 'Damion Lee': 20,
 'Nemanja Bjelica': 21,
 'Andre Iguodala': 22,
 'Juan Toscano-Anderson': 23,
 'Otto Porter Jr.': 24,
 'Moses Moody': 25,
 'Gary Payton II': 26,
 'Anthony Davis': 27,
 'LeBron James': 28,
 'Russell Westbrook': 29,
 'Kent Bazemore': 30,
 'DeAndre Jordan': 31,
 'Carmelo Anthony': 32,
 'Rajon Rondo': 33,
 'Malik Monk': 34,
 'Dwight Howard': 35,
 'Avery Bradley': 36,
 'Austin Reaves': 37,
 'Sekou Doumbouya': 38,
 'Giannis Antetokounmpo': 39,
 'Khris Middleton': 40,
 'Brook Lopez': 41,
 'Grayson Allen': 42,
 'Jrue Holiday': 43,
 'Pat Connaughton': 44,
 'Jord

In [15]:
conn.execute('''DROP TABLE player;''')
conn.execute('''CREATE TABLE player(
                PlayerId INTEGER NOT NULL,
                Name VARCHAR,
                PRIMARY KEY (PlayerId)
                )''')

for ele in player_dict:
    conn.execute( ''' INSERT INTO player(PlayerId, Name) VALUES(?,?) ''', (int(player_dict[ele]), ele))

In [16]:
results = list(conn.execute( ''' SELECT * FROM player'''))
print(results)

[(0, 'Joe Harris'), (1, 'James Harden'), (2, 'Kevin Durant'), (3, 'Nic Claxton'), (4, 'Blake Griffin'), (5, 'Patty Mills'), (6, 'James Johnson'), (7, 'Jevon Carter'), (8, 'LaMarcus Aldridge'), (9, 'Paul Millsap'), (10, "DeAndre' Bembry"), (11, 'Bruce Brown'), (12, 'Cam Thomas'), (13, 'Kessler Edwards'), (14, 'David Duke Jr.'), (15, 'Stephen Curry'), (16, 'Draymond Green'), (17, 'Andrew Wiggins'), (18, 'Jordan Poole'), (19, 'Kevon Looney'), (20, 'Damion Lee'), (21, 'Nemanja Bjelica'), (22, 'Andre Iguodala'), (23, 'Juan Toscano-Anderson'), (24, 'Otto Porter Jr.'), (25, 'Moses Moody'), (26, 'Gary Payton II'), (27, 'Anthony Davis'), (28, 'LeBron James'), (29, 'Russell Westbrook'), (30, 'Kent Bazemore'), (31, 'DeAndre Jordan'), (32, 'Carmelo Anthony'), (33, 'Rajon Rondo'), (34, 'Malik Monk'), (35, 'Dwight Howard'), (36, 'Avery Bradley'), (37, 'Austin Reaves'), (38, 'Sekou Doumbouya'), (39, 'Giannis Antetokounmpo'), (40, 'Khris Middleton'), (41, 'Brook Lopez'), (42, 'Grayson Allen'), (43, 'J

In [18]:
conn.commit()
conn.close()